In [ ]:
import sys 
import pandas as pd 
import numpy as np
import requests
import multiprocessing 
from multiprocessing import Pool
from itertools import repeat
import json
import sys

import re, numpy as np, pandas as pd
from pprint import pprint

import gensim, logging, warnings
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
from gensim.utils import simple_preprocess

from gensim.utils import simple_preprocess
import nltk
import spacy

nltk.download('stopwords')

In [ ]:
df = pd.read_excel('textfile.xlsx')
print (df.head())

In [ ]:
merged_series = df.apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
combined_text = ' '.join(merged_series)
print(combined_text)

In [ ]:
data = str(combined_text)
print(data)

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

stopwords_add= ['from', 'subject', 're', 'edu', 'use','also', 'well', 'including', 'simple', 'yet',  'work', 'inform',  'generally','given', 'work', 'help', 'play', 
                'et', 'al','across', 'towards','allow', 'body','affect','give', 'addtion','know', 'whose','real','open','often','although','act','find','even',
              'suggest','connect','tightly','release','space','function','regulate','control','role','apt','process', 'finely', 'tune','regulation','critical','functioning',
              'endeavor','finely','conditions','term','project','using','changes','fundamental', 'build','block','make','addition', 'provide', 'train','individual',
              'feasible', 'complex','mechanisms', 'remain', 'unclear', 'overarch','system', 'thereby', 'movement', 'valuable', 'educational','consider','waste',
              'study', 'field', 'establish','award','understand','expression','community','reach', 'the', 'on', 'in',
              'statutory','statutory','head','call','group','new','size','behavior','leave','internalize',
              'via','ecosystem','model','organisms','harsh','concurrently','precise','small','essential','long','release',
               'random','live','range','really','form','global','potential','collection','important','exposure','extreame','order', 'recipient','extreme','break','still','together',
              'cells','build','block', 'live','however','dont', 'exactly', 'cells', 'form', 'first', 'place', 'dont','molecules','cell', 'finally','question', 'challenge', 'pose', 'require',...]
stop_words.update(stopwords_add)
# stopwords.extend(stopwords_add)

%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [ ]:
def remove_stopwords(combined_text):
    import string
    # nltk.download('punkt')  # Download tokenizer if not already done
    text = combined_text.translate(str.maketrans('', '', string.punctuation)).lower()
    tokens = text.split()
    # tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return ' '.join(filtered_tokens)
filtered_text = remove_stopwords(combined_text)
print(filtered_text)

In [ ]:
from nltk.tokenize import WhitespaceTokenizer
tokenizer_w = WhitespaceTokenizer()
tokenized_text = tokenizer_w.tokenize(filtered_text)
print(tokenized_text)

In [ ]:
type(tokenized_text)

In [ ]:
token_clean_text = [re.sub("\'", "", str(tokenized_text)) for sent in tokenized_text]
print(token_clean_text[:1])

In [ ]:
token_clean_strg = str(token_clean_text)

In [ ]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
nltk.download ('punkt_tab')

stemmer = SnowballStemmer("english", True)
words = word_tokenize(token_clean_strg)
stemmed_words = [stemmer.stem(word) for word in words]

In [ ]:
stemmed_words_strg = str(stemmed_words)

In [ ]:
cleaned_texts = [sent.replace("'", "").replace(",", "").replace(":", "").replace(".", "").replace("(", "").replace(")", "") for sent in stemmed_words_strg]

In [ ]:
with open('output.txt', 'w', encoding='utf-8') as f:
     for item in cleaned_texts:         
         f.write(item)
print("File saved successfully as 'output.txt'")

In [ ]:
file_obj = open("output.txt", "r", encoding="utf-8")
print(file_obj.read(1000))

In [ ]:
cleaned_strg1 = ''.join(map(str,cleaned_texts))

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:         
        return wordnet.NOUN
       
def lemmatize_passage(cleaned_strg1):
    words = word_tokenize(cleaned_strg1)
    pos_tags = pos_tag(words)
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
    lemmatized_sentence = ' '.join(lemmatized_words)
    return lemmatized_sentence

result = lemmatize_passage(cleaned_strg1)

In [ ]:
with open('result.txt', 'w', encoding='utf-8') as f:
     for item in result:         
         f.write(item)
print("File saved successfully as 'result.txt'")

In [ ]:
def Convert(string):
    li = list(string.split(" "))
    return li
list_result = Convert(result)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             #stop_words='english',             # remove stop words
                             #lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}'  # num chars > 3
                             # max_features=50000,             # max number of unique words
                            )

data_vectorized = vectorizer.fit_transform(list_result)

In [ ]:
vector = data_vectorized

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix

def select_random_rows(matrix, num_rows):
    # Get the number of rows in the matrix
    total_rows = matrix.shape[0]
    
    # Generate random row indices
    random_indices = np.random.choice(total_rows, size=num_rows, replace=False)
    
    # Extract the selected rows
    selected_rows = matrix[random_indices]
    
    # Create a new csr_matrix from the selected rows
    new_matrix = csr_matrix(selected_rows)
    
    return new_matrix

num_rows_to_select = 1000000
new_matrix = select_random_rows(vector, num_rows_to_select)

print(new_matrix[:10])

In [ ]:
data_dense = new_matrix.todense()
print("Sparsity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
import spacy

In [ ]:
lda_model = LatentDirichletAllocation(n_components=10,            # Number of topics
                                      max_iter=5,                # Max learning iterations
                                      learning_method='online',  # The method used for learning 'online' indicates online variational Bayes
                                      #random_state=100,          # Random state When you set a specific value for random_state, you guarantee that the same data points will be included in the training and testing sets every time you run the code.
                                      batch_size=200,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: -1 skips the perplexity
                                      n_jobs = -1,               # Use all available CPUs; -1 uses all available cores
                                     )
lda_output = lda_model.fit_transform(new_matrix)

print(lda_model)

In [ ]:
print("Log Likelihood: ", lda_model.score(new_matrix))
print("Perplexity: ", lda_model.perplexity(new_matrix))
pprint(lda_model.get_params())

In [ ]:
search_params = {'n_components': [5, 8, 12, 16], 'learning_decay': [.5, .7, .9]}
lda = LatentDirichletAllocation()
model = GridSearchCV(lda, param_grid=search_params)
model.fit(new_matrix)

In [ ]:
best_lda_model = model.best_estimator_
print("Best Model's Params: ", model.best_params_)
print("Best Log Likelihood Score: ", model.best_score_)
print("Model Perplexity: ", best_lda_model.perplexity(new_matrix))

In [ ]:
df_topic_keywords = pd.DataFrame(best_lda_model.components_)
topicnames = [f'Topic {i}' for i in range(best_lda_model.n_components)]
df_topic_keywords.columns = vectorizer.get_feature_names_out()
df_topic_keywords.index = topicnames
df_topic_keywords

In [ ]:
df_topic_keywords.to_csv('LDAtopics.csv')